## 0. Dependency Libraries 

In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns

In [10]:
# Read the original dataset
data_dir = ''
file_path = 'screening_exercise_orders_v201810.csv'
ori_data = pd.read_csv(os.path.join(data_dir, file_path), header = 0)
ori_data.sample(5)

,customer_id,gender,date,value,predicted_gender
9466,7270,1,2017-09-06 21:00:40,776.8,0
9003,1427,0,2017-08-24 15:40:27,199.6,0
6189,5306,0,2017-06-04 18:55:59,233.2,0
11469,3389,1,2017-11-08 13:59:46,804.0,1
9966,7558,0,2017-09-22 14:43:44,178.0,0


In [28]:
# check the length first
print("Original dataset length: "+str(len(ori_data)))

Original dataset length: 13471


## A) Assemble a dataframe with one row per customer and the following columns:
#### * customer_id * gender * most_recent_order_date * order_count (number of orders placed by this customer)
#### Sort the dataframe by customer_id ascending and display the first 10 rows.

In [46]:
# Group rows by using customer_id and gender, and find the most_recent_order_date for each customer
customer_data = pd.DataFrame(ori_data.groupby(['customer_id', 'gender'], sort=False)['date'].max()).rename(columns={'date':'most_recent_order_date'})
customer_data.sample(3)

,,most_recent_order_date
customer_id,gender,
7081,0,2017-08-26 23:50:56
7049,1,2017-08-25 19:49:26
3956,0,2017-04-20 15:00:11


In [47]:
# check the length after grouping
print("Customer dataset length: "+str(len(tmp_data)))

Customer dataset length: 8814


In [48]:
# Add order_count for each customer
customer_data['order_count'] = ori_data['customer_id'].value_counts().sort_index().values
customer_data.sample(3)

,,most_recent_order_date,order_count
customer_id,gender,,
8197,1,2017-10-20 21:49:40,1
5179,1,2017-05-27 23:06:07,2
8786,1,2017-11-24 21:19:27,1


In [49]:
# Sort the dataframe by customer_id ascending and display the first 10 rows.
customer_data.head(10)

,,most_recent_order_date,order_count
customer_id,gender,,
1000,0,2017-01-01 00:11:31,1
1001,0,2017-01-01 00:29:56,1
1002,1,2017-02-19 21:35:31,3
1003,1,2017-04-26 02:37:20,4
1004,0,2017-01-01 03:11:54,1
1005,1,2017-12-16 01:39:27,2
1006,1,2017-05-09 15:27:20,3
1007,0,2017-01-01 15:59:50,1
1008,0,2017-12-17 05:47:48,3
